# Phase space study

This notebook reproduces the results shown in the paper. In src/utils.jl you can find several functions which allow to choose which portion of parameter space you want to explore , with or without prior parameter inference. In this example we use gam_vs_lam_learn which means that at fixed observations dilution, false rate, total time T, and graph ensemble (degree_dist) the space of gamma vs lambda is explored with the hyperparameters inferred. The function returns a huge matrix that is saved in the next two cells. REMARK. You have to create the folder in which the matrix is saved. See later to more information

In [ ]:
import OffsetArrays
using Plots
using Revise
using ProgressMeter
using SparseArrays, LinearAlgebra, IndexedGraphs, Graphs
using IterTools
using DelimitedFiles
include("../src/bp.jl") 
include("../src/post_infer.jl") 
include("../src/observables.jl") 
include("../src/utils.jl")

# Population dynamics for the messages

In [ ]:
#Set here the total epidemic time T, the observations dilution, the range of parameters. 

T = 8 # discrete time: total epidemic time T
N = 20000; #population size: keep fixed (or change it if you really want) 
fr = 0.0 # noise of measure
grid = 21 # the grid of the phase diagram. 
dilution = 9/10 #observations dilution: change it from 0 (all indiv. observed once) to 1 (no observations)
λRange = LinRange(0.05,0.5,grid) 
γRange = LinRange(0.05,0.5,grid) 

21-element LinRange{Float64, Int64}:
 0.05,0.0725,0.095,0.1175,0.14,0.1625,…,0.3875,0.41,0.4325,0.455,0.4775,0.5

In [ ]:

degree_dist = Poisson(3)
γi = 0.5
λi = 0.5
#here we choose what regime we want to study (λ VS dilution, λ VS γ, dilution VS fr ...)
# go to src/utils
inf_out = gam_vs_lam_learn(γRange, λRange, dilution, N, T, degree_dist, fr; λi = 0.5,γi = 0.5, tot_iterations = 20, count_obs = false)

In [ ]:
# Here we write the results in files.
samp = 15
graph = "er"

In [ ]:
open("./dil_lam_$(graph)_$(samp)/ninf.txt","w") do io
    writedlm(io,inf_out[:,:,1]) 
end
for t = 1:T+1
    open("./dil_lam_$(graph)_$(samp)/AUC$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1]) 
    end
    open("./dil_lam_$(graph)_$(samp)/Ov$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1 + T + 1]) 
    end
    open("./dil_lam_$(graph)_$(samp)/L1_$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1 + 2*T + 2]) 
    end
    open("./dil_lam_$(graph)_$(samp)/MSE$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1 + 3*T + 3]) 
    end
end
open("./dil_lam_$(graph)_$(samp)/Fe.txt","w") do io
    writedlm(io,inf_out[:,:,end]) 
end